# Get the dataset

In [3]:
import numpy as np
import config

dataset = (np.load(config.x_path, mmap_mode="r"), np.load(config.y_path, mmap_mode="r"))
print(dataset[0].shape, dataset[1].shape)
assert(dataset[0].shape[0] == dataset[1].shape[0])


(15702, 3, 224, 224) (15702,)


# Boilerplate Code

In [5]:
from sklearn.model_selection import KFold
import config

kfold = KFold(config.kfold_nsplits, shuffle=True, random_state=config.seed)


In [8]:
from typing import Tuple
import torch

class Data(torch.utils.data.Dataset):
    def __init__(self, data: Tuple[np.ndarray, np.ndarray]) -> None:
        self.data = data
        assert(data[0].shape[0] == data[1].shape[0])
    
    def __iter__(self):
        for x, y in self.data:
            yield x, y
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx: int):
        return self.data[0][idx], self.data[1][idx]


# Model

In [11]:
from torch import nn
from torchvision.models import vgg16, VGG16_Weights
use_cuda = torch.cuda.is_available()
if not use_cuda:
    print("CUDA not used!")
device = torch.device("cuda" if use_cuda else "cpu")
model = vgg16(weights=VGG16_Weights.DEFAULT, progress=True)
model.classifier[6] = nn.Linear(4096, 2)



VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1